In [1]:
# Load libraries
import pandas as pd
import os
import statsmodels.api as sm

In [2]:
# Load cleaned data
listings_df = pd.read_csv(os.path.join('data', 'yvr_listing_data_cleaned.csv'))

In [8]:
"""
Using the logistic regression Wald test, see if there is a connection between review_scores_rating (independent variable)
and legal_listing (dependent variable). The null hypothesis is that there is no connection between the two variables.
review_scores_rating is a continuous variable from 0 to 5, while legal_listing is a categorial boolean variable.
"""
# Remove rows where review_scores_rating is NaN
listings_df = listings_df[listings_df['review_scores_rating'].notna()]

# Prepare the data
X = listings_df['review_scores_rating']
y = listings_df['legal_listing']

# Fit the logistic regression model
model = sm.Logit(y, sm.add_constant(X))
result = model.fit()

print(result.summary())

# Perform the Wald test
wald_test = result.wald_test("review_scores_rating = 0", scalar=True)

# Print the Wald test results
print("\n       Wald Test Results")
print("Test Statistic:", wald_test.statistic)
print("P-value:", wald_test.pvalue)
print("Degrees of Freedom:", wald_test.df_denom)


Optimization terminated successfully.
         Current function value: 0.563565
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          legal_listing   No. Observations:                 5641
Model:                          Logit   Df Residuals:                     5639
Method:                           MLE   Df Model:                            1
Date:                Tue, 21 Nov 2023   Pseudo R-squ.:                0.003890
Time:                        18:36:54   Log-Likelihood:                -3179.1
converged:                       True   LL-Null:                       -3191.5
Covariance Type:            nonrobust   LLR p-value:                 6.257e-07
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.4340      0.302     -1.437      0.151      -1.026       0.158